In [ ]:
import torch
from torch import nn
from torchvision import transforms, datasets
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import random_split, DataLoader
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from tqdm import tqdm
import numpy as np

In [ ]:
# Добавим в модель softmax, и укажем необходимое количество классов

class Model(nn.Module):
    def __init__(self, num_classes = 3):
        super().__init__()
        self.resnet = resnet18(weights = ResNet18_Weights.DEFAULT)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(in_features, num_classes),
            nn.Softmax())
    def forward(self, x):
        return self.resnet(x)

In [ ]:
# загружаем модель
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = Model().to(device)
model

Model(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [ ]:
# если код ниже не работает, запусти это

!rm -R ./train/ipynb_checkpoints
!ls ./train/ -a

rm: cannot remove './train/ipynb_checkpoints': No such file or directory
ls: cannot access './train/': No such file or directory


In [ ]:
# загружаем данные

transform = transforms.Compose([transforms.RandomRotation(0.3),
                                transforms.ToTensor(),
                                transforms.Resize((224,224), antialias = True),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                               ])

dataset = datasets.ImageFolder(root = './train', transform = transform)
num_classe = len(dataset.classes)
print("Classes: ", dataset.classes)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train, val = torch.utils.data.random_split(dataset, [train_size, val_size])

train_dataloader = torch.utils.data.DataLoader(dataset = train,
                                           batch_size = 32,
                                           shuffle = True)

val_dataloader = torch.utils.data.DataLoader(dataset = val,
                                         batch_size = 32,
                                         shuffle = True)

Classes:  ['left', 'other', 'right']


In [ ]:
# обучаем

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 50
best_f1 = 0

for epoch in range(num_epochs):
        train_bar = tqdm(train_dataloader, total=len(train_dataloader))
        model.train()
        print(f'Epoch: {epoch+1}, num epochs: {num_epochs}')
        for step, batch in enumerate(train_bar):
            images = batch[0].to(device)
            labels = batch[1].to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            train_bar.set_description("epoch {} loss {}".format(epoch,loss))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        val_bar = tqdm(val_dataloader,total=len(val_dataloader))
        model.eval()
        logits = []
        y_trues = []
        eval_loss = 0
        for step, batch in enumerate(val_bar):
            with torch.no_grad():
                images = batch[0].to(device)
                labels = batch[1].to(device)
                outputs = model(images)
                val_loss = criterion(outputs, labels)
                logits.append(outputs.cpu().numpy())
                y_trues.append(labels.cpu().numpy())
        logits = np.concatenate(logits,0)
        y_trues = np.concatenate(y_trues,0)
        best_threshold = 0.5
        y_preds = logits[:,1]>best_threshold
        recall = recall_score(y_trues, y_preds, average = "macro")
        precision = precision_score(y_trues, y_preds, average = "macro")
        f1 = f1_score(y_trues, y_preds, average = "macro")
        result = {
            "eval_recall": float(recall),
            "eval_precision": float(precision),
            "eval_f1": float(f1),
            "eval_threshold": best_threshold,
            "eval_loss": eval_loss
        }

        print("***** Eval results *****")
        for key in  sorted(result.keys()):
            print(key, str(round(result[key],4)))
        if result['eval_f1']>best_f1:
            best_f1=result['eval_f1']
            print("  "+"*"*20)
            print("  Best f1:",round(best_f1,4))
            print("  "+"*"*20)



  0%|          | 0/1 [00:00<?, ?it/s]

Epoch: 1, num epochs: 50


100%|██████████| 1/1 [00:00<00:00,  1.12it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


***** Eval results *****
eval_f1 0.4
eval_loss 0
eval_precision 0.3333
eval_recall 0.5
eval_threshold 0.5
  ********************
  Best f1: 0.4
  ********************


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


...

***** Eval results *****
eval_f1 0.1667
eval_loss 0
eval_precision 0.1111
eval_recall 0.3333
eval_threshold 0.5


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch: 10, num epochs: 50


epoch 9 loss 0.5522676706314087:   0%|          | 0/1 [00:07<?, ?it/s]


KeyboardInterrupt: ignored